In [1]:
%cd ~/project/team/code

/opt/ml/project/team/code


In [2]:
from data_utils import (WOSDataset, load_dataset,
                        seed_everything)
from evaluation import _evaluation

from train_loop import trade_train_loop, submt_train_loop
from inference import trade_inference, sumbt_inference 

from prepare import get_data, get_stuff, get_model

from attrdict import AttrDict

In [116]:
args = AttrDict(
    data_dir= '/opt/ml/input/data/train_dataset',
    ontology_root='/opt/ml/input/data/train_dataset/edit_ontology_metro.json',
    use_domain_slot='basic',
    
    use_small_data=False,
    train_batch_size=8,
    model_name_or_path= 'dsksd/bert-ko-small-minimal',
    vocab_size=35000,
    preprocessor='TRADEPreprocessor',
    model_class='TRADE',
    ModelName='TRADE',
    use_convert_ont=False,
    use_sys_usr_sys_turn=False,
    train_from_trained=None,
)

model_folder = 'results/decoder_long/'
model_name = 'model-best.bin'

In [117]:
data, slot_meta, ontology = get_data(args)
train_data, _, _, _ = load_dataset(data, 0)

## 하나 고르세요.

In [136]:
def filter_by_domain(data, domain):
    found_idx = set()
    for i in range(len(data)):
        if domain in data[i]['domains']:
            found_idx.add(i)
    return [data[i] for i in found_idx]

domain = '택시'
small_data = filter_by_domain(train_data, domain)

In [7]:
def filter_by_domain_slot(data, target):
    found_idx = set()
    for i in range(len(data)):
        find_next = False
        for dig in data[i]['dialogue']:
            if 'state' in dig:
                for st in dig['state']:
                    domain, slot, value = st.split('-')
                    if f'{domain}-{slot}' == target:
                        find_next = True
                        found_idx.add(i)
                        break
            if find_next:
                break
    return [data[i] for i in found_idx]

domain_slot = '식당-예약 시간'
small_data = filter_by_domain_slot(train_data, domain_slot)

In [118]:
def filter_by_value(data, target):
    found_idx = set()
    for i in range(len(data)):
        find_next = False
        for dig in data[i]['dialogue']:
            if 'state' in dig:
                for st in dig['state']:
                    domain, slot, value =  st.split('-')
                    if value == target:
                        find_next = True
                        found_idx.add(i)
                        break
            if find_next:
                break
    return [data[i] for i in found_idx]

target = '그레이 호텔'
small_data = filter_by_value(train_data, target)

In [137]:
print(len(small_data))
tokenizer, processor, train_features, dev_features = get_stuff(args,
                 small_data, None, slot_meta, ontology)

2374


Token indices sequence length is longer than the specified maximum sequence length for this model (537 > 512). Running this sequence through the model will result in indexing errors


In [138]:
import json
import argparse
import torch
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler


tokenized_slot_meta = []
for slot in slot_meta:
    tokenized_slot_meta.append(
        tokenizer.encode(slot.replace("-", " "), add_special_tokens=False)
    )
config = json.load(open(f"{model_folder}/exp_config.json", "r"))
config = argparse.Namespace(**config)
model =  get_model(config, tokenizer, ontology, slot_meta)
ckpt = torch.load(f'{model_folder}/{model_name}', map_location="cpu")
model.load_state_dict(ckpt)

train_data2 = WOSDataset(train_features)
train_sampler = RandomSampler(train_data2)
train_loader = DataLoader(
    train_data2,
    batch_size=args.train_batch_size,
    sampler=train_sampler,
    collate_fn=processor.collate_fn,
)

## 일단 TRADE 모델 꺼

In [139]:
def postprocess_state(state):
    for i, s in enumerate(state):
        s = s.replace(" : ", ":")
        s = s.replace(" & ", "&")
        s = s.replace(" = ", "=")
        s = s.replace("( ", "(")
        s = s.replace(" )", ")")
        state[i] = s.replace(" , ", ", ")
    return state

In [140]:
device = torch.device('cpu') # 개인 사정에 맞게 (cpu | cuda)
model.to(device)
print()

In [ ]:
from tqdm.auto import tqdm
from training_recorder import RunningLossRecorder
from torch.cuda.amp import autocast
use_amp = False

model.eval()
predictions = {}
no_post = {}
eval_loader = train_loader
pbar = tqdm(eval_loader, total=len(eval_loader))
loss_recorder = RunningLossRecorder(len(eval_loader))
cnt = 0
for batch in pbar:
    input_ids, segment_ids, input_masks, gating_ids, target_ids, guids = [
        b.to(torch.device(device)) if not isinstance(b, list) else b for b in batch
    ]

    with torch.no_grad():
        with autocast(enabled=use_amp):
            o, g = model(input_ids, segment_ids, input_masks, 9)

        _, generated_ids = o.max(-1)
        _, gated_ids = g.max(-1)


    for guid, gate, gen in zip(guids, gated_ids.tolist(), generated_ids.tolist()):
        prediction = processor.recover_state(gate, gen)
        no_post[guid] = prediction
        prediction = postprocess_state(prediction)
        predictions[guid] = prediction

    cnt += 1
#     if cnt == 1:
#         break
    
pbar.close()


In [ ]:
from data_utils import get_examples_from_dialogues
user_first = False
dialogue_level = False
processor_kwargs = AttrDict()
    
train_examples = get_examples_from_dialogues(
    small_data, user_first=user_first, dialogue_level=dialogue_level,use_sys_usr_sys=True,
        which='train'
)

## 보는 방법은 CONTROL + ENTER로

* MISSED: 정답인데 예측 하지 않은 거
*  WRONG: 예측한 값이 틀린 경우
* CORREC: 제대로 예측한 경우

In [131]:
idx = 0
ln = len(train_examples)

In [135]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
while idx < ln:
    datum = train_examples[idx]
    idx += 1
    guid = datum.guid
    answer = set(datum.label)
    pred = set(predictions[guid])
    if set(datum.label) != set(predictions[guid]):
        print(f'dialogue {idx}')
        print(f'guid: {guid}')
        for i in range(len(datum.context_turns)//2):
            print(f'SYS: {datum.context_turns[2*i]}')
            print(f'USR: {datum.context_turns[2*i + 1]}')

        print()
        print(f'SYS: {datum.current_turn[0]}')
        print(f'USR: {datum.current_turn[1]}')
        if True:#args.use_sys_usr_sys_turn:
            print(f'SYS: {datum.current_turn[2]}')
        print()
        tab = '\t'
        print(f'MISSED: {tab.join(sorted(answer - pred))}')
        print(f'WRONG : {tab.join(sorted(pred - answer))}')
        print(f'CORREC: ')
        pp.pprint(sorted(answer & pred))
        break
 

유저가 제안(No State) -> 시스템 inform(yes State)

In [ ]:
user_suggested = 

유저가 제안(Yes State)

유저가 요청(No State) -> 시스템 inform(yes State)

유저가 요청(Yes State) -> 시스템 inform